In [1]:
# load in the tab delimited file
import pandas as pd

myfile = "logfile_20190428_1914.tsv"
df = pd.read_csv(myfile,sep=' ',header = None)


In [2]:
# create a list of dates 
from datetime import date, timedelta,datetime

d1 = datetime(2019, 1, 1,0,0)  # start date
d2 = datetime(2019, 3,31 ,0,0)  # end date

delta = d2 - d1         # timedelta

dateRecord=[]
for i in range(delta.days + 1):
    dates = d1 + timedelta(days=i)
    
    # first must convert to string format before append
    dates = '{}'.format(dates)   
    #print(dates)
    dateRecord.append(dates)
print(dateRecord)


['2019-01-01 00:00:00', '2019-01-02 00:00:00', '2019-01-03 00:00:00', '2019-01-04 00:00:00', '2019-01-05 00:00:00', '2019-01-06 00:00:00', '2019-01-07 00:00:00', '2019-01-08 00:00:00', '2019-01-09 00:00:00', '2019-01-10 00:00:00', '2019-01-11 00:00:00', '2019-01-12 00:00:00', '2019-01-13 00:00:00', '2019-01-14 00:00:00', '2019-01-15 00:00:00', '2019-01-16 00:00:00', '2019-01-17 00:00:00', '2019-01-18 00:00:00', '2019-01-19 00:00:00', '2019-01-20 00:00:00', '2019-01-21 00:00:00', '2019-01-22 00:00:00', '2019-01-23 00:00:00', '2019-01-24 00:00:00', '2019-01-25 00:00:00', '2019-01-26 00:00:00', '2019-01-27 00:00:00', '2019-01-28 00:00:00', '2019-01-29 00:00:00', '2019-01-30 00:00:00', '2019-01-31 00:00:00', '2019-02-01 00:00:00', '2019-02-02 00:00:00', '2019-02-03 00:00:00', '2019-02-04 00:00:00', '2019-02-05 00:00:00', '2019-02-06 00:00:00', '2019-02-07 00:00:00', '2019-02-08 00:00:00', '2019-02-09 00:00:00', '2019-02-10 00:00:00', '2019-02-11 00:00:00', '2019-02-12 00:00:00', '2019-02-1

In [3]:
# check number of unique IP addresses
len(df.iloc[:,3].unique())
IP_list = df.iloc[:,3].unique().tolist()
type(IP_list)

list

In [4]:
# make daily temperature out of Mean daily temperature data set
# use statistical knowledge of inverse Norm , and then add the mean to the value to create inverse norm on non standard normal dist
# first call in the mean average daily temperature to use the mean value

# there is mean max and mean min temp, so i want to find the average of these 2
df_max = pd.read_csv("weather/surface-air-temperature-monthly-mean-daily-maximum.csv")
df_min = pd.read_csv("weather/surface-air-temperature-monthly-mean-daily-minimum.csv")
df_rainfall = pd.read_csv("weather/rainfall-monthly-number-of-rain-days.csv")
# since the log file only from Jan 2019 to March 2019, we take the mean only for these 3 months
df_max_short = df_max.loc[444:446,:]
df_min_short = df_min.loc[444:446,:]
df_rainfall_short = df_rainfall.loc[444:446,:] # index see from the df printed out not from excel
                  
# reindexing since the index become 444 to 446
df_min_short = df_min_short.reset_index()
del df_min_short['index']
df_max_short = df_max_short.reset_index()
del df_max_short['index']
df_rainfall_short = df_rainfall_short.reset_index()
del df_rainfall_short['index']
df_rainfall_short.no_of_rainy_days[2]                          
df_max_short

,month,temp_mean_daily_max
0,2019-01,31.7
1,2019-02,32.8
2,2019-03,33.5


In [5]:
# create random average temperature for each day 
import random
numDays = len(dateRecord)

# times 10 to the temperature since randint can only take integers, then divide back to become 1 dec place num
temp = []
for item in dateRecord:
    month = int(item[6])  
    #print(month)
    #print(monthIndex)
    if month == 1:
        #a = df_min_short.iloc[month][1]
        #print(a)
        #print()
        #b = df_rainfall_short.iloc[month]
        #print(b)
        #print()
        #c = random.randint(1,5)
        #print(c)
        temp.append((random.randint((df_min_short.iloc[month-1][1])*10,(df_max_short.iloc[month-1][1])*10))/10)
    if month == 2:
        temp.append((random.randint((df_min_short.iloc[month-1][1])*10,(df_max_short.iloc[month-1][1])*10))/10)
    if month == 3:
        temp.append((random.randint((df_min_short.iloc[month-1][1])*10,(df_max_short.iloc[month-1][1])*10))/10)

temp[1:10]


[26.4, 25.7, 26.7, 26.3, 28.3, 25.7, 31.4, 29.8, 31.3]

In [6]:
# fix a threshold for good weather and threshold for bad weather
# if a day got rain is also Bad weather
# https://smallbusiness.chron.com/randomize-list-python-26724.html
thresholdHighestTemp = 31 # degree

# generate a index so i can choose exactly the number of times it rains in the mth (based on data.gov.sg)
# but make the position of 1 is random
index = list(range(len(df_rainfall)))
# generate random index
randomIdx = []
index_Jan = list(range(1,32))
index_Feb = list(range(1,29))
index_Mar = list(range(1,32))

#shuffle then remove last few elements corr to num of days raining that month
random.shuffle(index_Jan) 
random.shuffle(index_Feb)
random.shuffle(index_Mar) 
print(index_Jan)
print(index_Feb)
print(index_Mar)
#print(len(index_Mar))

# drop the number of values corr by number of days it rain in that month
del index_Jan[-(df_rainfall_short.no_of_rainy_days[0]) :]
print("after deleted Jan {} : ".format(index_Jan))
del index_Feb[-(df_rainfall_short.no_of_rainy_days[1]) :]
print("after deleted Feb {} : ".format(index_Feb))
del index_Mar[-(df_rainfall_short.no_of_rainy_days[2]) :]
print("after deleted Mar {} : ".format(index_Mar))
print(len(index_Jan),len(index_Feb),len(index_Mar))

randomList = [
    index_Jan ,
    index_Feb ,
    index_Mar ,
]
print(randomList)

[27, 11, 14, 7, 3, 25, 24, 22, 9, 21, 31, 2, 8, 16, 20, 29, 30, 5, 13, 26, 28, 18, 19, 1, 15, 23, 10, 17, 6, 12, 4]
[13, 26, 9, 28, 22, 18, 7, 25, 27, 23, 10, 14, 21, 3, 11, 24, 1, 20, 4, 16, 17, 12, 6, 19, 8, 2, 5, 15]
[27, 6, 17, 26, 24, 25, 5, 2, 31, 10, 20, 15, 11, 14, 19, 12, 23, 28, 4, 21, 9, 3, 29, 18, 16, 1, 13, 30, 22, 8, 7]
after deleted Jan [27, 11, 14, 7, 3, 25, 24, 22, 9, 21, 31, 2, 8, 16, 20, 29, 30, 5, 13, 26, 28, 18] : 
after deleted Feb [13, 26, 9, 28, 22, 18, 7, 25, 27, 23, 10, 14, 21, 3, 11, 24, 1, 20, 4, 16, 17, 12, 6] : 
after deleted Mar [27, 6, 17, 26, 24, 25, 5, 2, 31, 10, 20, 15, 11, 14, 19, 12, 23, 28, 4, 21, 9, 3, 29, 18, 16] : 
22 23 25
[[27, 11, 14, 7, 3, 25, 24, 22, 9, 21, 31, 2, 8, 16, 20, 29, 30, 5, 13, 26, 28, 18], [13, 26, 9, 28, 22, 18, 7, 25, 27, 23, 10, 14, 21, 3, 11, 24, 1, 20, 4, 16, 17, 12, 6], [27, 6, 17, 26, 24, 25, 5, 2, 31, 10, 20, 15, 11, 14, 19, 12, 23, 28, 4, 21, 9, 3, 29, 18, 16]]


In [7]:
# simulate number of days of rain
# times 10 to the temperature since randint can only take integers, then divide back to become 1 dec place num
import numpy as np
from numpy import ones
binaryRain = []
janRain = ones([31])
febRain = ones([28])
marRain = ones([31])

mthRain = [
            ones([31]), #jan
            ones([28]), #feb
            ones([31]), #mar
          ]

# replace 1 for days that rain. 0 for days that did not rain
def replaceOnes(listed,monthrain):
    for index in listed:
        monthrain[index-1]=0 # position index is -1 of the day
    return(monthrain)

for i in range(len(mthRain)):
    mthRain[i] = replaceOnes(randomList[i], mthRain[i]) #mthRain[0] is JanRain
print(mthRain)
mthRain_toDf = np.concatenate(mthRain).ravel().tolist()
mthRain_toDf[1:10]
len(mthRain_toDf) #check there are correct number of days betw Jan to March

[array([1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.])]


90

In [8]:
# make to a dataframe from the lists above
d = {'date':dateRecord,'temperature':temp, 'rainyDays':mthRain_toDf}
df = pd.DataFrame(d)
df.head()
type(df.rainyDays[0])

numpy.float64

In [9]:
# create data sales online on rainy days vs non rainy days
# create data sales online on a very hot day (above 31 deg) vs good weather
from random import randint
sales = []
for i in range(0,len(df)):
    if df.temperature[i] > 31 and df.rainyDays[i] == 0:
        expSales = randint(100,800)
        sales.append(expSales)
    elif df.temperature[i] <= 31 and df.rainyDays[i] == 0:
        expSales = randint(100,600)
        sales.append(expSales)
    elif df.temperature[i] > 31 and df.rainyDays[i] == 1:
        expSales = randint(300,800)
        sales.append(expSales)
    else:
        expSales = randint(100,600)
        sales.append(expSales)
sales[1:10]

[405, 181, 397, 154, 308, 569, 377, 384, 623]

In [10]:
# finally, append this last column to the df
df = df.assign(sales_in_dollars = sales)
df.to_csv("bonusData1_salesWeather.csv")
df.head()

,date,temperature,rainyDays,sales_in_dollars
0,2019-01-01 00:00:00,25.6,1.0,459
1,2019-01-02 00:00:00,26.4,0.0,405
2,2019-01-03 00:00:00,25.7,0.0,181
3,2019-01-04 00:00:00,26.7,1.0,397
4,2019-01-05 00:00:00,26.3,0.0,154
